In [ ]:
import pandas as pd

In [ ]:
import tkinter as tk
from tkinter import filedialog

In [ ]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

In [ ]:
df = pd.read_excel(file_path, sheet_name="出貨明細")

In [ ]:
df = df[(df["狀態"].str.contains("出")) & (df["預交日期"] <= "2021-01-31")]

In [ ]:
monthly_sales = df.groupby([pd.Grouper(key="預交日期", freq="M"), "負責業務"])[["數量", "本國幣別NTD"]].sum().reset_index()

In [ ]:
monthly_sales

In [ ]:
format_dict = {"預交日期": "{:%Y-%m}", "數量": "{0:,.0f}", "本國幣別NTD": "{0:,.0f}"} # 設定樣式字典來套用
test = monthly_sales.style.format(format_dict).hide_index().background_gradient(subset=["數量", "本國幣別NTD"], cmap="BuGn")

In [ ]:
monthly_sales.style.format(format_dict).hide_index().bar(color="lightgreen", vmin=100_000, subset=["本國幣別NTD"]).set_caption("2019年至今各月統計表")

In [ ]:
month_order = ["January", "February", "March", "April",
              "May", "June", "July", "August",
              "September", "October", "November", "December"]
df["預交月份"] = pd.Categorical(df["預交月份"], ordered=True, categories=month_order)

In [ ]:
result = test.data # .data將styler轉回dataframe才能進行其他操作，例如作圖之類的

## Pivot table轉回DataFrame方式為
---
1. 使用.to_records()將數據轉為array
2. 在用pd.DataFrame的方式將array放進去即可

In [ ]:
pd.DataFrame(pd.pivot_table(df, index=["預交年份", "負責業務"], values="本國幣別NTD", columns="預交月份").to_records())

In [ ]:
import plotly.express as px

In [ ]:
px.line(result, x="預交日期", y="本國幣別NTD", color="負責業務", 
        title="業績走勢圖", labels={"本國幣別NTD": "營業額", "預交日期": "月份"}).show()

In [ ]:
test.to_excel(r"C:\Users\kaihsu\Desktop\test.xlsx", index=False)